In [36]:
import pandas as pd
import numpy as np
# import datetime
from datetime import datetime, timezone
import requests
import json
import matplotlib.pyplot as plt
import math
# import scipy.stats as st

In [37]:
def create_tuple(response):
    # all data
    response_data = response.json()
    # print(response_data)
    # data for a single target
    response_datum = response_data[0]
    # print(response_datum)
    # the target - e.g. "120MapleAve.STM.M22-V/AverageMassFlow"
    response_target = response_datum['target']
    # the metric for this target - e.g. "Steam"
    #response_metric = response_datum['metric']
    # print(response_metric)
    # the datapoints as a list of lists
    response_datapoints = response_datum['datapoints']
    # print(response_datapoints)
    # array of tuples for numpy
    tuple_array = [tuple(x) for x in response_datapoints]
    # make a numpy array
    npa = np.array(tuple_array, dtype=[
                   ('value', np.double), ('ts', 'datetime64[ms]')])
    # npa = npa.astype([('value', np.double), ('ts', datetime)])

    # print("here")
    # for index, i in enumerate(npa):
    #   j = pd.to_datetime(i[1]).replace(tzinfo=timezone.utc).astimezone(tz="America/New_York")
    #   npa[index][1] = j
    return npa

In [38]:
def query():
  url = "https://portal.emcs.cornell.edu/api/datasources/proxy/5/query"
  data = {
      "range": {
        "from": "2022-12-25T00:00:00.000",
        "to": "2022-12-30T00:00:00.000",
      },
      "targets": [
        {
          "target": "AliceCookHouse.HW.FP/TONS"
        }
      ],

    }
  request = requests.post(url, json=data)
  print(request)
  # print(request.json())
  return create_tuple(request)

In [39]:
hw_master = query()
hw_list = pd.Series(data=[i[0] for i in hw_master], index=[i[1] for i in hw_master])
hw_list = hw_list[~hw_list.index.duplicated()]
print("CFM List: ", hw_list)

<Response [200]>
CFM List:  2022-12-25 00:00:00    18650.4750
2022-12-25 01:00:00    18432.4250
2022-12-25 02:00:00    17815.7000
2022-12-25 03:00:00    18280.2000
2022-12-25 04:00:00    17624.9250
                          ...    
2022-12-29 20:00:00     7556.9850
2022-12-29 21:00:00     7083.5375
2022-12-29 22:00:00     8893.0325
2022-12-29 23:00:00     9366.6175
2022-12-30 00:00:00     8862.2400
Length: 121, dtype: float64


In [32]:
df = pd.DataFrame(hw_list)
df.columns=["data"]
df

,data
2022-12-25 00:00:00,18650.4750
2022-12-25 01:00:00,18432.4250
2022-12-25 02:00:00,17815.7000
2022-12-25 03:00:00,18280.2000
2022-12-25 04:00:00,17624.9250
...,...
2022-12-29 20:00:00,7556.9850
2022-12-29 21:00:00,7083.5375
2022-12-29 22:00:00,8893.0325
2022-12-29 23:00:00,9366.6175


In [35]:
df[df["data"] < 10000]

,data
2022-12-28 18:00:00,9534.7300
2022-12-29 15:00:00,9287.7700
2022-12-29 16:00:00,8427.7200
2022-12-29 17:00:00,8185.0400
2022-12-29 18:00:00,7747.5100
2022-12-29 19:00:00,6853.8175
2022-12-29 20:00:00,7556.9850
2022-12-29 21:00:00,7083.5375
2022-12-29 22:00:00,8893.0325
2022-12-29 23:00:00,9366.6175
